In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

In [2]:
vaccine_symp = "data/vaccine_symptoms.csv"
vaccine_symp_df = pd.read_csv(vaccine_symp)
vaccine_symp_df.head()

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,916710,Appendicitis,23.1,Band neutrophil percentage increased,23.1,Surgery,23.1,White blood cell count increased,23.1,NaN,NaN
1,916720,Dysphagia,23.1,Dyspnoea,23.1,NaN,NaN,NaN,NaN,NaN,NaN
2,916741,Chills,23.1,Complex regional pain syndrome,23.1,Fatigue,23.1,Headache,23.1,Joint range of motion decreased,23.1
3,916741,Myalgia,23.1,Pain in extremity,23.1,Peripheral swelling,23.1,X-ray abnormal,23.1,NaN,NaN
4,916772,Feeling abnormal,23.1,SARS-CoV-2 test positive,23.1,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
vaccine_type = "data/vaccine_type.csv"
vaccine_type_df = pd.read_csv(vaccine_type)
vaccine_type_df.head()

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,916710,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
1,916720,COVID19,MODERNA,011L20A,1,IM,AR,COVID19 (COVID19 (MODERNA))
2,916741,COVID19,PFIZER\BIONTECH,EH9899,1,SYR,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
3,916772,COVID19,PFIZER\BIONTECH,EJ1685,UNK,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
4,916790,COVID19,PFIZER\BIONTECH,NaN,1,IM,RA,COVID19 (COVID19 (PFIZER-BIONTECH))


In [4]:
vaccine_data = "data/vaccine_data.csv"
vaccine_df = pd.read_csv(vaccine_data)
vaccine_df.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 612: invalid start byte

In [7]:
vaccine_type_cols = ["VAERS_ID", "VAX_TYPE", "VAX_MANU", "VAX_DOSE_SERIES", "VAX_ROUTE", "VAX_NAME"]
vaccine_type_transformed= vaccine_type_df[vaccine_type_cols].copy()

vaccine_type_transformed = vaccine_type_transformed.rename(columns={"VAERS_ID":"vaers_id", "VAX_TYPE": "vax_type",
                                                                    "VAX_MANU":"vax_manu", "VAX_DOSE_SERIES":"vax_dose_series",
                                                                    "VAX_ROUTE":"vax_route", "VAX_NAME":"vax_name"})

vaccine_type_transformed.drop_duplicates("vaers_id", inplace=True)
vaccine_type_transformed.set_index("vaers_id", inplace=True)

vaccine_type_transformed.head()

,vax_type,vax_manu,vax_dose_series,vax_route,vax_name
vaers_id,,,,,
916710,COVID19,MODERNA,1,IM,COVID19 (COVID19 (MODERNA))
916720,COVID19,MODERNA,1,IM,COVID19 (COVID19 (MODERNA))
916741,COVID19,PFIZER\BIONTECH,1,SYR,COVID19 (COVID19 (PFIZER-BIONTECH))
916772,COVID19,PFIZER\BIONTECH,UNK,IM,COVID19 (COVID19 (PFIZER-BIONTECH))
916790,COVID19,PFIZER\BIONTECH,1,IM,COVID19 (COVID19 (PFIZER-BIONTECH))


In [8]:
vaccine_symp_cols = ["VAERS_ID", "SYMPTOM1", "SYMPTOM2", "SYMPTOM3", "SYMPTOM4", "SYMPTOM5"]
vaccine_symp_transformed= vaccine_symp_df[vaccine_symp_cols].copy()

vaccine_symp_transformed = vaccine_symp_transformed.rename(columns={"VAERS_ID":"vaers_id", "SYMPTOM1": "symptom1",
                                                                    "SYMPTOM2":"symptom2", "SYMPTOM3":"symptom3",
                                                                    "SYMPTOM4":"symptom4", "SYMPTOM5":"symptom5"})

vaccine_symp_transformed.drop_duplicates("vaers_id", inplace=True)
vaccine_symp_transformed.set_index("vaers_id", inplace=True)

vaccine_symp_transformed.head()

,symptom1,symptom2,symptom3,symptom4,symptom5
vaers_id,,,,,
916710,Appendicitis,Band neutrophil percentage increased,Surgery,White blood cell count increased,NaN
916720,Dysphagia,Dyspnoea,NaN,NaN,NaN
916741,Chills,Complex regional pain syndrome,Fatigue,Headache,Joint range of motion decreased
916772,Feeling abnormal,SARS-CoV-2 test positive,NaN,NaN,NaN
916790,Blood pressure diastolic increased,Blood pressure systolic increased,Chest discomfort,Dizziness,Dysphagia


In [9]:
connection_string = "postgres:postgras@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
engine.table_names()

['vaccine_type', 'vaccine_symp']

In [12]:
vaccine_symp_transformed.to_sql(name='vaccine_symp', con=engine, if_exists='append', index=True)

In [15]:
vaccine_type_transformed.to_sql(name='vaccine_type', con=engine, if_exists='append', index=True)